### Create Daily Input Data

In [0]:
from pyspark.sql import functions as F


In [0]:
appointments = spark.createDataFrame([
    ("Ramesh K", "9XXXXXXXXX", "OPD", "Viral Fever Meds", "2024-08-05"),
    ("Sita P", "9XXXXXXXXX", "Procedure", "Minor Skin Procedure", "2024-08-07"),
    ("Arjun M", "9XXXXXXXXX", "OPD", "Gastric Medication", None)
], ["patient_name", "phone", "visit_type", "prescription", "follow_up_date"])


### Message Type Classification (One-Time Master Table)

In [0]:
message_type_master = spark.createDataFrame([
    ("Follow-up Reminder", False),
    ("Post-Procedure Care", False),
    ("Side-Effect Advisory", False),
    ("Custom Instruction", True),
    ("Patient Question Response", True)
], ["message_type", "doctor_input_required"])


### Generate Care_Control Sheet 

#### Auto-assign Message Type

In [0]:
care_control = (
    appointments
    .withColumn(
        "message_type",
        F.when(F.col("follow_up_date").isNotNull(), "Follow-up Reminder")
         .when(F.col("visit_type") == "Procedure", "Post-Procedure Care")
         .otherwise("Custom Instruction")
    )
)


#### Join Doctor Approval Logic

In [0]:
care_control = (
    care_control
    .join(message_type_master, "message_type", "left")
    .withColumn(
        "doctor_approval",
        F.when(F.col("doctor_input_required"), "Required")
         .otherwise("Not Required")
    )
    .withColumn(
        "status",
        F.when(F.col("doctor_input_required"), "Waiting")
         .otherwise("Pending")
    )
    .withColumn("message_text", F.lit(None))
)


In [0]:
care_control.show()


### Doctor Review Window (Exception Filter)

In [0]:
doctor_review_queue = care_control.filter(
    (F.col("doctor_approval") == "Required") &
    (F.col("status") == "Waiting")
)

doctor_review_queue.show()


### Message Dispatch Logic

In [0]:
dispatch_ready = care_control.filter(
    (F.col("message_text").isNotNull()) &
    (F.col("status") == "Pending") &
    (
        (F.col("doctor_approval") == "Not Required") |
        (F.col("doctor_approval") == "Approved")
    )
)

dispatch_ready.show()


### Daily Closing Check

In [0]:
pending_end_of_day = care_control.filter(F.col("status") == "Pending")
pending_end_of_day.show()


### Automation Readiness

In [0]:
care_control = care_control.withColumn(
    "status",
    F.when(F.col("status") == "Pending", "Sent").otherwise(F.col("status"))
)
